# 02. Advanced Ollama Setup

## 1. Introduction

In this notebook, we'll set up multiple Ollama instances in Docker containers. Ollama is an open-source tool that allows us to run large language models locally. This setup will enable us to use different models for various tasks in our RAG (Retrieval-Augmented Generation) system, such as embedding generation and text generation.

We'll cover the following steps:
1. Updating our project directory structure
2. Creating an OllamaManager class to handle Ollama operations
3. Updating our environment variables
4. Updating our Docker Compose configuration
5. Testing the OllamaManager class

## 2. Update Project Directory Structure

First, let's update our project directory structure to accommodate the Ollama models and ensure they're shared between containers.

In [ ]:
import os

# Get the project root directory
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Create directories for Ollama models
ollama_models_path = os.path.join(project_root, 'db_data', 'ollama_models')
ollama_llm_path = os.path.join(ollama_models_path, 'llm')

os.makedirs(ollama_models_path, exist_ok=True)
os.makedirs(ollama_llm_path, exist_ok=True)

print(f"Created Ollama models directory at: {ollama_models_path}")
print(f"Created Ollama LLM directory at: {ollama_llm_path}")


Our updated project structure now looks like this:

```
RAG_tools/
├── config/
│   ├── docker-compose.yml
│   └── .env
├── notebooks/
│   ├── 00_Environment_Setup.ipynb
│   ├── 01_Database_Setup.ipynb
│   └── 02_Ollama_setup.ipynb
├── src/
│   └── utils/
│       ├── config_utils.py
│       └── ollama_manager.py
├── db_data/
│   ├── postgres/
│   ├── neo4j/
│   └── ollama_models/
│       └── llm/
└── tests/
```

## 3. Create OllamaManager Class

Now, let's create the OllamaManager class. This class will be used to launch and manage Ollama instances in Docker containers.

In [ ]:
%%writefile ../src/utils/ollama_manager.py
import os
import requests
import time
import logging
import json
from .DockerComposeManager import DockerComposeManager
from .config_utils import Config

class OllamaManager:
    def __init__(self, config: Config):
        self.config = config
        self.container_name = self.config.OLLAMA_LLM_CONTAINER_NAME
        self.port = self.config.OLLAMA_LLM_PORT
        self.model = self.config.OLLAMA_LLM_MODEL
        self.gpu = self.config.OLLAMA_LLM_GPU
        
        self.models_path = self.config.OLLAMA_MODELS_PATH or os.path.expanduser('~/ollama_models')
        self.llm_path = self.config.OLLAMA_LLM_PATH or os.path.join(self.models_path, 'llm')
        
        # Ensure directories exist
        os.makedirs(self.models_path, exist_ok=True)
        os.makedirs(self.llm_path, exist_ok=True)
        
        # Initialize DockerComposeManager
        docker_compose_path = os.path.join('..', 'config', 'docker-compose.yml')
        self.docker_manager = DockerComposeManager(docker_compose_path)

        logging.info(f"OllamaManager initialized with models_path: {self.models_path}, llm_path: {self.llm_path}")
        logging.info(f"Using model: {self.model} on port: {self.port}")

    def ensure_model_available(self):
        if not self.is_model_running():
            logging.info(f"Model {self.model} not found. Attempting to download...")
            self.pull_model()
        else:
            logging.info(f"Model {self.model} is already available.")

    def generate_response(self, prompt):
        self.ensure_model_available()
        try:
            payload = {
                'model': self.model,
                'prompt': prompt
            }
            logging.debug(f"Sending request to Ollama API with payload: {payload}")
            logging.debug(f"API URL: http://localhost:{self.port}/api/generate")
            
            response = requests.post(
                f'http://localhost:{self.port}/api/generate',
                json=payload,
                stream=True
            )
            logging.debug(f"Response status code: {response.status_code}")
            response.raise_for_status()
            
            full_response = ""
            for line in response.iter_lines():
                if line:
                    try:
                        chunk = json.loads(line)
                        logging.debug(f"Received chunk: {chunk}")
                        if 'response' in chunk:
                            token = chunk['response']
                            full_response += token
                            print(token, end='', flush=True)
                        if chunk.get('done', False):
                            break
                    except json.JSONDecodeError:
                        logging.warning(f"Failed to decode JSON: {line}")
            
            print("\n")  # New line after the response
            return full_response.strip()
        except requests.exceptions.RequestException as e:
            logging.error(f"Error generating response: {str(e)}")
            if hasattr(e, 'response') and e.response is not None:
                logging.error(f"Response content: {e.response.text}")
            return f"Error: {str(e)}"
        except Exception as e:
            logging.error(f"Unexpected error: {str(e)}")
            return f"Unexpected error: {str(e)}"

    def is_model_running(self):
        try:
            response = requests.get(f'http://localhost:{self.port}/api/tags')
            response.raise_for_status()
            models = response.json()
            logging.debug(f"Available models: {models}")
            return self.model in [model['name'] for model in models['models']]
        except requests.exceptions.RequestException as e:
            logging.error(f"Error checking if model is running: {e}")
            return False

    def pull_model(self):
        logging.info(f"Pulling model {self.model}...")
        try:
            response = requests.post(f'http://localhost:{self.port}/api/pull', json={'name': self.model}, stream=True)
            response.raise_for_status()
            for line in response.iter_lines():
                if line:
                    print(line.decode())
            logging.info(f"Successfully pulled model {self.model}")
        except requests.exceptions.RequestException as e:
            logging.error(f"Error pulling model: {str(e)}")
            raise RuntimeError(f"Failed to download model {self.model}. Error: {str(e)}")

    def start_container(self):
        self.docker_manager.start_containers()
        logging.info(f"Started container: {self.container_name}")
        self.wait_for_ollama()
        self.ensure_model_available()

    def stop_container(self):
        self.docker_manager.stop_containers()
        logging.info(f"Stopped container: {self.container_name}")

    def wait_for_ollama(self, max_attempts=5, delay=5):
        for attempt in range(max_attempts):
            try:
                response = requests.get(f'http://localhost:{self.port}/api/tags')
                if response.status_code == 200:
                    logging.info(f"Successfully connected to Ollama on port {self.port}")
                    return True
            except requests.exceptions.RequestException:
                logging.warning(f"Attempt {attempt + 1}/{max_attempts}: Ollama on port {self.port} is not ready yet. Retrying in {delay} seconds...")
                time.sleep(delay)
        logging.error(f"Failed to connect to Ollama after {max_attempts} attempts")
        raise RuntimeError(f"Failed to connect to Ollama after {max_attempts} attempts")


## 4. Update Environment Variables

Now, let's update our .env file to include the Ollama-related variables:


In [ ]:
import os

env_file_path = os.path.join('..', 'config', '.env')

ollama_env_vars = """
# Ollama Configuration
OLLAMA_EMBEDDING_CONTAINER_NAME=ragtools_ollama_embedding
OLLAMA_EMBEDDING_PORT=12434
OLLAMA_EMBEDDING_MODEL=bert-base-multilingual-cased
OLLAMA_EMBEDDING_GPU=0

OLLAMA_LLM_CONTAINER_NAME=ragtools_ollama_llm
OLLAMA_LLM_PORT=12435
OLLAMA_LLM_MODEL=tinyllama
OLLAMA_LLM_GPU=1

OLLAMA_MODELS_PATH=./db_data/ollama_models
OLLAMA_LLM_PATH=./db_data/ollama_models/llm
"""

with open(env_file_path, 'a') as f:
    f.write(ollama_env_vars)

print("Updated .env file with Ollama configurations.")


## 5. Update Config Class

Before we proceed with the verification step, we need to update our Config class to include the new Ollama-related attributes. This is a crucial step when extending our framework with new components.

This step demonstrates how to extend the Config class when new components are added to the framework. It's important to update this class whenever new environment variables or configuration options are introduced.

Let's update the `config_utils.py` file:

In [ ]:
import os

config_utils_path = os.path.join('..', 'src', 'utils', 'config_utils.py')

# Read the existing content
with open(config_utils_path, 'r') as f:
    existing_content = f.read()

# Define the new Ollama configurations
ollama_configs = '''        # Ollama configurations
        self.OLLAMA_EMBEDDING_CONTAINER_NAME = os.getenv('OLLAMA_EMBEDDING_CONTAINER_NAME')
        self.OLLAMA_EMBEDDING_PORT = int(os.getenv('OLLAMA_EMBEDDING_PORT', 12434))
        self.OLLAMA_EMBEDDING_MODEL = os.getenv('OLLAMA_EMBEDDING_MODEL')
        self.OLLAMA_EMBEDDING_GPU = int(os.getenv('OLLAMA_EMBEDDING_GPU', 0))
        self.OLLAMA_LLM_CONTAINER_NAME = os.getenv('OLLAMA_LLM_CONTAINER_NAME')
        self.OLLAMA_LLM_PORT = int(os.getenv('OLLAMA_LLM_PORT', 12435))
        self.OLLAMA_LLM_MODEL = os.getenv('OLLAMA_LLM_MODEL')
        self.OLLAMA_LLM_GPU = int(os.getenv('OLLAMA_LLM_GPU', 1))
        self.OLLAMA_MODELS_PATH = os.getenv('OLLAMA_MODELS_PATH')
        self.OLLAMA_LLM_PATH = os.getenv('OLLAMA_LLM_PATH')
'''

# Find the position to insert the new configurations
lines = existing_content.split('\n')
insert_line = -1
for i, line in enumerate(lines):
    if line.strip().startswith('def get_postgres_connection_params(self):'):
        insert_line = i
        break

if insert_line == -1:
    # If method not found, insert at the end of __init__
    for i, line in enumerate(reversed(lines)):
        if line.strip() == "self.DOCKER_NETWORK_NAME = os.getenv('DOCKER_NETWORK_NAME')":
            insert_line = len(lines) - i
            break

# Insert the new configurations
if insert_line != -1:
    updated_lines = lines[:insert_line] + ollama_configs.split('\n') + lines[insert_line:]
    updated_content = '\n'.join(updated_lines)
else:
    print("Could not find appropriate insertion point. Please update manually.")
    updated_content = existing_content

# Write the updated content back to the file
with open(config_utils_path, 'w') as f:
    f.write(updated_content)

print("Updated config_utils.py with Ollama configurations.")


## 6. Update Docker Compose Configuration

Let's update our docker-compose.yml file to include the Ollama services:

In [ ]:
import os
import yaml

docker_compose_path = os.path.join('..', 'config', 'docker-compose.yml')

# Read existing docker-compose.yml
with open(docker_compose_path, 'r') as f:
    docker_compose = yaml.safe_load(f)

# Add or update Ollama services
docker_compose['services']['ollama_embedding'] = {
    'image': 'ollama/ollama',
    'container_name': '${OLLAMA_EMBEDDING_CONTAINER_NAME}',
    'environment': ['OLLAMA_HOST=0.0.0.0:${OLLAMA_EMBEDDING_PORT}'],
    'ports': ['${OLLAMA_EMBEDDING_PORT}:${OLLAMA_EMBEDDING_PORT}'],
    'volumes': ['${OLLAMA_MODELS_PATH}:/root/.ollama'],
    'deploy': {
        'resources': {
            'reservations': {
                'devices': [{'driver': 'nvidia', 'count': 1, 'capabilities': ['gpu']}]
            }
        }
    },
    'networks': ['ragtools_network']
}

docker_compose['services']['ollama_llm'] = {
    'image': 'ollama/ollama',
    'container_name': '${OLLAMA_LLM_CONTAINER_NAME}',
    'environment': ['OLLAMA_HOST=0.0.0.0:${OLLAMA_LLM_PORT}'],
    'ports': ['${OLLAMA_LLM_PORT}:${OLLAMA_LLM_PORT}'],
    'volumes': [
        '${OLLAMA_MODELS_PATH}:/root/.ollama',
        '${OLLAMA_LLM_PATH}:/root/.ollama/llm'
    ],
    'deploy': {
        'resources': {
            'reservations': {
                'devices': [{'driver': 'nvidia', 'count': 1, 'capabilities': ['gpu']}]
            }
        }
    },
    'networks': ['ragtools_network']
}

# Ensure the network is defined
if 'networks' not in docker_compose:
    docker_compose['networks'] = {}
docker_compose['networks']['ragtools_network'] = {'name': '${DOCKER_NETWORK_NAME}'}

# Write updated docker-compose.yml
with open(docker_compose_path, 'w') as f:
    yaml.dump(docker_compose, f)

print("Updated docker-compose.yml with Ollama services.")


## 7. Test OllamaManager Class

Now, let's test our OllamaManager class by spinning up a test model and running a simple prompt:

In [ ]:
import sys
import os
import time
import logging

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

# Add the project root to the Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(project_root)

from src.utils.config_utils import Config
from src.utils.ollama_manager import OllamaManager
from src.utils.DockerComposeManager import DockerComposeManager

# Initialize the Config class
config = Config()

# Print all configuration values for debugging
print("Configuration values:")
for attr, value in vars(config).items():
    print(f"{attr}: {value}")

# Initialize DockerComposeManager
docker_compose_path = os.path.join(project_root, 'config', 'docker-compose.yml')

# Initialize OllamaManager for the embedding model
try:
    embedding_manager = OllamaManager(
        container_name=config.OLLAMA_EMBEDDING_0_CONTAINER_NAME,
        port=config.OLLAMA_EMBEDDING_0_PORT,
        model=config.OLLAMA_EMBEDDING_0_MODEL,
        gpu=config.OLLAMA_EMBEDDING_0_GPU,
        models_path=config.OLLAMA_MODELS_PATH,
        llm_path=config.OLLAMA_LLM_PATH,
        docker_compose_path=docker_compose_path
    )

    # Initialize OllamaManager for the Codestral model
    codestral_manager = OllamaManager(
        container_name=config.OLLAMA_CODESTRAL_CONTAINER_NAME,
        port=config.OLLAMA_CODESTRAL_PORT,
        model=config.OLLAMA_CODESTRAL_MODEL,
        gpu=config.OLLAMA_CODESTRAL_GPU,
        models_path=config.OLLAMA_MODELS_PATH,
        llm_path=config.OLLAMA_LLM_PATH,
        docker_compose_path=docker_compose_path
    )
except Exception as e:
    logging.error(f"Error initializing OllamaManager: {str(e)}")
    sys.exit(1)

# Start the containers
print("Starting Ollama containers...")
try:
    embedding_manager.start_container()
    codestral_manager.start_container()
except Exception as e:
    logging.error(f"Error starting containers: {str(e)}")
    sys.exit(1)

# Wait for containers to start
time.sleep(10)

# Check container status
print("\nChecking container status:")
docker_manager = DockerComposeManager(docker_compose_path)
docker_manager.show_container_status()

# Verify that models are available
print("\nVerifying model availability:")
embedding_manager.ensure_model_available()
codestral_manager.ensure_model_available()

# Test questions
test_questions = [
    "What is the capital of Texas?",
    "Write a Python function to calculate the Fibonacci sequence."
]

# Test the codestral model
print("\nTesting codestral model:")
for question in test_questions:
    print(f"\nPrompt: {question}")
    response = codestral_manager.generate_response(question)
    print(f"Response: {response}")

# Test the embedding model (if applicable)
# Note: This is just a placeholder. Adjust based on your embedding model's capabilities.
print("\nTesting Embedding model:")
embedding_text = "This is a test sentence for embedding."
print(f"Embedding text: {embedding_text}")
# Uncomment and adjust the following line based on your embedding model's interface
# embedding = embedding_manager.generate_embedding(embedding_text)
# print(f"Embedding (first 5 dimensions): {embedding[:5]}")

# Stop the containers
print("\nStopping Ollama containers...")
embedding_manager.stop_container()
codestral_manager.stop_container()

print("\nVerification complete. All containers have been stopped.")

# Additional debug information
print("\nDebug Information:")
print(f"OLLAMA_EMBEDDING_0_PORT: {config.OLLAMA_EMBEDDING_0_PORT}")
print(f"OLLAMA_EMBEDDING_0_MODEL: {config.OLLAMA_EMBEDDING_0_MODEL}")
print(f"OLLAMA_CODESTRAL_PORT: {config.OLLAMA_CODESTRAL_PORT}")
print(f"OLLAMA_CODESTRAL_MODEL: {config.OLLAMA_CODESTRAL_MODEL}")


## Conclusion

In this notebook, we have successfully:

1. Set up Ollama instances in Docker containers
2. Created an OllamaManager class to handle Ollama operations
3. Implemented a method to generate responses from the LLM
4. Demonstrated the streaming nature of the LLM's output
5. Verified the functionality of our setup with test questions

## Next Steps

Our next notebook will focus on creating a CLI interface for interacting with the LLM. Before diving into the implementation, we'll need to consider:

1. LLM Configurables:
   - Context length
   - Temperature
   - Other relevant parameters (e.g., top_p, frequency_penalty, presence_penalty)

2. CLI Interface Options:
   - Evaluate the merits of adopting a pre-built CLI interface vs. creating our own
   - Consider libraries like `click`, `typer`, or `argparse` for building a custom CLI

3. Chat Interface Design:
   - How to maintain conversation history
   - Handling user input and system responses
   - Implementing commands for adjusting LLM parameters on-the-fly

4. Integration with OllamaManager:
   - How to incorporate our existing OllamaManager class into the CLI interface

By addressing these points, we'll be well-prepared to create a robust and user-friendly CLI for interacting with our LLM setup.